# Clonado de repositorio y carga de datos

In [2]:
# MODO EXAMEN: clonado y carga de datos (sin rutas absolutas)
!test -d PIA_04_datasets || git clone https://github.com/kachytronico/PIA_04_datasets
!find PIA_04_datasets -name "datasets.zip"
!mkdir -p PIA_04_datasets/unzip
!unzip -o PIA_04_datasets/datasets.zip -d PIA_04_datasets/unzip
!find PIA_04_datasets/unzip -name "*.csv"



"test" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
fatal: destination path 'PIA_04_datasets' already exists and is not an empty directory.
Acceso denegado: PIA_04_DATASETS
No se encuentra el archivo: -NAME
La sintaxis del comando no es correcta.
"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
No se encuentra el archivo: PIA_04_DATASETS/UNZIP
No se encuentra el archivo: -NAME


## Carga de datos

### Objetivo y plan

Quiero localizar el CSV real de Tesla y cargarlo en `df_tesla` sin rutas absolutas.
Voy a revisar `df_tesla.shape` y un `head()` para validar filas, columnas y formato.

In [ ]:
import pandas as pd

# Aquí localizo y cargo solo Tesla
df_tesla = pd.read_csv("PIA_04_datasets/unzip/sistema_de_arranque.csv")# Se añaden comillas a la ruta para que sea un string válido
print("✅ df_tesla:", df_tesla.shape)
df_tesla.head()# Aquí reviso el CSV cargado


### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

# Problema 1: Sistema de arranque Tesla

## Realiza un AED sobre el conjunto de datos.

### Objetivo y plan

Quiero tomar una foto rapida del dataset para entender dimensiones y calidad.
Voy a mirar `df_tesla.shape`, un `head(3)` y el porcentaje de nulos para decidir donde limpiar primero.

In [ ]:
# Aquí hago un vistazo general del dataset
print("Filas, columnas:", df_tesla.shape)
display(df_tesla.head(3))

nulos = (df_tesla.isna().mean().sort_values(ascending=False) * 100).round(2)
display(nulos.to_frame("porcentaje_nulos_%").head(15))

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Estadísticos iniciales.

### Objetivo y plan

Quiero resumir rapidamente cada variable para ver rangos, dispersion y cardinalidad.
Con `describe(include="all")` espero detectar columnas con pocos valores unicos o rangos anomalo.

In [ ]:
# Aquí calculo estadísticos iniciales
display(df_tesla.describe(include="all").T)

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Distribuciones de las variables numéricas del conjunto de datos.

### Objetivo y plan

Quiero ver la forma de las variables numericas para detectar sesgos y colas largas.
Mirare cuantas columnas hay en `num_cols` y los histogramas para decidir si escalar o transformar.

In [ ]:
# Aquí visualizo distribuciones de variables numéricas
import matplotlib.pyplot as plt

num_cols = df_tesla.select_dtypes(include="number").columns
if len(num_cols) == 0:
    print("No hay columnas numéricas para graficar")
else:
    df_tesla[num_cols].hist(bins=20, figsize=(12, 8))
    plt.suptitle("Distribuciones numéricas")
    plt.tight_layout()
    plt.show()

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Matriz de correlación.

### Objetivo y plan

Quiero identificar relaciones fuertes entre variables numericas antes de modelar.
Voy a mirar el heatmap y la matriz `corr` para localizar correlaciones altas o redundancias.

In [ ]:
# Aquí calculo y muestro la matriz de correlación
import seaborn as sns
import matplotlib.pyplot as plt

num_cols = df_tesla.select_dtypes(include="number").columns
if len(num_cols) < 2:
    print("No hay suficientes columnas numéricas para correlación")
else:
    corr = df_tesla[num_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, cmap="coolwarm", center=0)
    plt.title("Matriz de correlación")
    plt.tight_layout()
    plt.show()

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Realiza el preprocesamiento de datos de tu problema.

### Objetivo y plan

Quiero definir el preprocesado completo sin fuga de datos.
Voy a dejar claros los pasos de imputacion, codificacion, escalado y PCA, ajustando solo con `X_train`.

In [ ]:
# TODO: plan de preprocesado (split, nulos, encoding, escalado, PCA)

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Reserva un conjunto de datos para validación y otro para testeo.

### Objetivo y plan

Quiero separar train, valid y test con estratificacion para mantener la proporcion de la clase objetivo.
Revisare tamanos y distribuciones para asegurar un split equilibrado.

In [ ]:
# TODO: train/valid/test con estratificación y tamaños

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Columnas inútiles, valores sin sentido y atípicos.

### Objetivo y plan

Quiero detectar columnas inutiles (IDs, constantes) y valores atipicos que distorsionen el modelo.
Me fijare en cardinalidad, varianza y reglas simples de rango sobre `X_train`.

In [ ]:
# TODO: criterios de limpieza en train (IDs, constantes, outliers)

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Tratamiento de valores nulos.

### Objetivo y plan

Quiero tratar los nulos con un imputador ajustado solo en `X_train`.
Voy a comparar el porcentaje de nulos antes y despues para comprobar el efecto.

In [ ]:
# TODO: imputación fit solo en train

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Análisis de variabilidad.

### Objetivo y plan

Quiero eliminar variables con varianza nula o casi nula para simplificar el modelo.
Voy a contabilizar cuantas columnas se descartan y cuantas quedan activas.

In [ ]:
# TODO: eliminar varianza 0/casi 0

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Columnas categóricas.

### Objetivo y plan

Quiero codificar las variables categoricas con OneHotEncoder.
Verificare cuantas columnas nuevas aparecen y el impacto en la dimensionalidad.

In [ ]:
# TODO: encoding categóricas (OneHotEncoder)

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Reducción de la dimensionalidad.

### Objetivo y plan

Quiero reducir dimensionalidad con PCA y conservar la mayor varianza posible.
Voy a revisar la varianza explicada y el numero de componentes elegidos.

In [ ]:
# TODO: PCA (fit solo en train)

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Entrena y optimiza un modelo de KNN.

### Entrenar un KNN.

### Objetivo y plan

Quiero entrenar un KNN base como punto de partida.
Voy a medir su rendimiento en validacion para comparar con la optimizacion.

In [ ]:
# TODO: entrenamiento KNN

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

### Optimizar el KNN con la técnica GridSearch.

### Objetivo y plan

Quiero optimizar el KNN con GridSearch para encontrar el mejor `n_neighbors` y pesos.
Usare el score de validacion para elegir el modelo final.

In [ ]:
# TODO: GridSearchCV para KNN

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Entrena y optimiza un modelo de DT.

### Entrenar un DT.

### Objetivo y plan

Quiero entrenar un arbol de decision base para tener una referencia clara.
Medire su rendimiento y complejidad antes de optimizar hiperparametros.

In [ ]:
# TODO: entrenamiento DT

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

### Explicar el DT.

### Objetivo y plan

Quiero explicar el arbol antes de optimizarlo para entender que variables mandan.
Revisare `feature_importances_` y la profundidad del arbol.

In [ ]:
# TODO: explicación DT (antes)

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

### Optimizar el DT con la técnica RandomSearch.

### Objetivo y plan

Quiero buscar un mejor DT con RandomizedSearchCV sin sobreajustar.
Voy a comparar el mejor score con el modelo base para ver la mejora real.

In [ ]:
# TODO: RandomizedSearchCV para DT

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

### Explicar el DT.

### Objetivo y plan

Quiero explicar el DT optimizado y compararlo con el arbol inicial.
Mirare cambios en importancias, profundidad y numero de hojas.

In [ ]:
# TODO: explicación DT (después)

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Entrena y optimiza un modelo de SVM.

### Entrenar un SVM.

### Objetivo y plan

Quiero entrenar un SVM base para tener una referencia solida.
Revisare el rendimiento y el numero de vectores soporte.

In [ ]:
# TODO: entrenamiento SVM

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

### Optimizar el SVM con la técnica GridSearch.

### Objetivo y plan

Quiero optimizar el SVM probando kernel, `C` y `gamma`.
Usare el mejor score de validacion para elegir el modelo final.

In [ ]:
# TODO: GridSearchCV para SVM

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Entrena y optimiza un modelo de NL.

### Entrenar un NL.

### Objetivo y plan

Quiero entrenar un MLPClassifier base con escalado previo.
Medire su rendimiento y el numero de iteraciones hasta converger.

In [ ]:
# TODO: entrenamiento NL (MLPClassifier)

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

### Optimizar el NL con la técnica RandomSearch.

### Objetivo y plan

Quiero optimizar la red probando tamanos de capa, activacion y regularizacion.
Escogere la configuracion con mejor score de validacion.

In [ ]:
# TODO: RandomizedSearchCV para NL

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

## Crear modelos ensemble usando:

### Los tres mejores modelos obtenidos, usando el siguiente criterio: media aritmética de todos los modelos que tengan una fiabilidad superior al 80%.

### Objetivo y plan

Quiero combinar los tres mejores modelos usando solo predicciones con fiabilidad > 0.80.
Si ninguno supera el umbral, definire un fallback con el mejor modelo individual.

In [ ]:
# TODO: ensemble por fiabilidad > 0.80

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.

### Todos los modelos, usando como criterio un modelo de Regresión Lineal.

### Objetivo y plan

Quiero crear un ensemble con regresion lineal usando las probabilidades de todos los modelos.
Definire un umbral de decision en base a la metrica de validacion.

In [ ]:
# TODO: ensemble con LinearRegression

### Conclusiones (a completar tras ejecutar)Aqui completare los valores observados en el output (shape, % nulos, correlaciones).Incluire las metricas clave y, si aplica, los mejores hiperparametros.Comparare con modelos o ensembles para justificar la decision final.